# Análisis de Regresión Lineal - E-commerce Expenses

**Autor:** Agustín Villarreal  
**Programa:** Maestría en Cómputo Aplicado  
**Email:** agustin.villarreal0743@alumnos.udg.mx  
**Fecha:** Octubre 2025

---

## Objetivo
Analizar el impacto de diferentes variables en el gasto total (Total Spend) de transacciones de e-commerce, específicamente:
1. Evaluar si la variable "Record" mejora la bondad de ajuste del modelo
2. Identificar variables con impacto negativo en las predicciones

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## 1. Carga y Exploración de Datos

In [ ]:
# Cargar el dataset
df = pd.read_csv('Ecom Expense.csv')

# Visualizar primeras filas
print("Primeras 5 filas del dataset:")
print(df.head())

print("\nInformación general del dataset:")
print(df.info())

print("\nEstadísticas descriptivas:")
print(df.describe())

In [ ]:
# Verificar valores nulos
print("Valores nulos por columna:")
print(df.isnull().sum())

# Verificar duplicados
print(f"\nRegistros duplicados: {df.duplicated().sum()}")

## 2. Análisis Exploratorio de Datos (EDA)

In [ ]:
# Análisis de correlación con todas las variables numéricas
numerical_cols = ['Age ', 'Items ', 'Monthly Income', 'Transaction Time', 'Record', 'Total Spend']
# Nota: hay espacios en algunos nombres de columnas

# Renombrar columnas para eliminar espacios
df.columns = df.columns.str.strip()

plt.figure(figsize=(12, 8))
correlation_matrix = df[['Age', 'Items', 'Monthly Income', 'Transaction Time', 'Record', 'Total Spend']].corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación - Variables Numéricas', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nCorrelación con Total Spend:")
print(correlation_matrix['Total Spend'].sort_values(ascending=False))

In [ ]:
# Distribución de la variable objetivo
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df['Total Spend'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Total Spend')
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribución de Total Spend')
axes[0].grid(alpha=0.3)

axes[1].boxplot(df['Total Spend'])
axes[1].set_ylabel('Total Spend')
axes[1].set_title('Boxplot de Total Spend')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Análisis de la variable Record
print("Análisis de la variable 'Record':")
print(f"Valores únicos: {df['Record'].unique()}")
print(f"\nDistribución de Record:")
print(df['Record'].value_counts().sort_index())

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

df['Record'].value_counts().sort_index().plot(kind='bar', ax=axes[0], edgecolor='black')
axes[0].set_xlabel('Record')
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribución de la Variable Record')
axes[0].grid(alpha=0.3)

df.groupby('Record')['Total Spend'].mean().plot(kind='bar', ax=axes[1], edgecolor='black')
axes[1].set_xlabel('Record')
axes[1].set_ylabel('Total Spend Promedio')
axes[1].set_title('Total Spend Promedio por Record')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Preparación de Datos

In [ ]:
# Codificar variables categóricas
le_gender = LabelEncoder()
le_city = LabelEncoder()

df['Gender_Encoded'] = le_gender.fit_transform(df['Gender'])
df['City_Tier_Encoded'] = le_city.fit_transform(df['City Tier'])

print("Codificación de variables categóricas:")
print(f"Gender: {dict(zip(le_gender.classes_, le_gender.transform(le_gender.classes_)))}")
print(f"City Tier: {dict(zip(le_city.classes_, le_city.transform(le_city.classes_)))}")

## 4. Modelo Base (SIN la variable Record)

In [ ]:
# Definir features sin Record
features_sin_record = ['Age', 'Items', 'Monthly Income', 'Transaction Time', 
                       'Gender_Encoded', 'City_Tier_Encoded']

X_sin_record = df[features_sin_record]
y = df['Total Spend']

# División de datos
X_train_sr, X_test_sr, y_train, y_test = train_test_split(
    X_sin_record, y, test_size=0.2, random_state=42
)

# Entrenar modelo
modelo_sin_record = LinearRegression()
modelo_sin_record.fit(X_train_sr, y_train)

# Predicciones
y_pred_train_sr = modelo_sin_record.predict(X_train_sr)
y_pred_test_sr = modelo_sin_record.predict(X_test_sr)

# Métricas
print("="*60)
print("MODELO SIN LA VARIABLE RECORD")
print("="*60)
print("\nMétricas en conjunto de ENTRENAMIENTO:")
print(f"R² Score: {r2_score(y_train, y_pred_train_sr):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train_sr)):.4f}")
print(f"MAE: {mean_absolute_error(y_train, y_pred_train_sr):.4f}")

print("\nMétricas en conjunto de PRUEBA:")
r2_sin_record = r2_score(y_test, y_pred_test_sr)
rmse_sin_record = np.sqrt(mean_squared_error(y_test, y_pred_test_sr))
mae_sin_record = mean_absolute_error(y_test, y_pred_test_sr)

print(f"R² Score: {r2_sin_record:.4f}")
print(f"RMSE: {rmse_sin_record:.4f}")
print(f"MAE: {mae_sin_record:.4f}")

In [ ]:
# Análisis de coeficientes - Modelo sin Record
coef_df_sr = pd.DataFrame({
    'Feature': features_sin_record,
    'Coefficient': modelo_sin_record.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print("\nCoeficientes del modelo (ordenados por valor absoluto):")
print(coef_df_sr)
print(f"\nIntercepto: {modelo_sin_record.intercept_:.4f}")

# Visualización
plt.figure(figsize=(10, 6))
colors = ['red' if x < 0 else 'green' for x in coef_df_sr['Coefficient']]
plt.barh(coef_df_sr['Feature'], coef_df_sr['Coefficient'], color=colors, edgecolor='black')
plt.xlabel('Coeficiente')
plt.title('Coeficientes del Modelo SIN Record', fontweight='bold')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

## 5. Modelo CON la variable Record

In [ ]:
# Definir features con Record
features_con_record = ['Age', 'Items', 'Monthly Income', 'Transaction Time', 
                       'Record', 'Gender_Encoded', 'City_Tier_Encoded']

X_con_record = df[features_con_record]

# División de datos
X_train_cr, X_test_cr, y_train, y_test = train_test_split(
    X_con_record, y, test_size=0.2, random_state=42
)

# Entrenar modelo
modelo_con_record = LinearRegression()
modelo_con_record.fit(X_train_cr, y_train)

# Predicciones
y_pred_train_cr = modelo_con_record.predict(X_train_cr)
y_pred_test_cr = modelo_con_record.predict(X_test_cr)

# Métricas
print("="*60)
print("MODELO CON LA VARIABLE RECORD")
print("="*60)
print("\nMétricas en conjunto de ENTRENAMIENTO:")
print(f"R² Score: {r2_score(y_train, y_pred_train_cr):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train_cr)):.4f}")
print(f"MAE: {mean_absolute_error(y_train, y_pred_train_cr):.4f}")

print("\nMétricas en conjunto de PRUEBA:")
r2_con_record = r2_score(y_test, y_pred_test_cr)
rmse_con_record = np.sqrt(mean_squared_error(y_test, y_pred_test_cr))
mae_con_record = mean_absolute_error(y_test, y_pred_test_cr)

print(f"R² Score: {r2_con_record:.4f}")
print(f"RMSE: {rmse_con_record:.4f}")
print(f"MAE: {mae_con_record:.4f}")

In [ ]:
# Análisis de coeficientes - Modelo con Record
coef_df_cr = pd.DataFrame({
    'Feature': features_con_record,
    'Coefficient': modelo_con_record.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print("\nCoeficientes del modelo (ordenados por valor absoluto):")
print(coef_df_cr)
print(f"\nIntercepto: {modelo_con_record.intercept_:.4f}")

# Visualización
plt.figure(figsize=(10, 6))
colors = ['red' if x < 0 else 'green' for x in coef_df_cr['Coefficient']]
plt.barh(coef_df_cr['Feature'], coef_df_cr['Coefficient'], color=colors, edgecolor='black')
plt.xlabel('Coeficiente')
plt.title('Coeficientes del Modelo CON Record', fontweight='bold')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

## 6. Comparación de Modelos

In [ ]:
# Tabla comparativa
comparacion = pd.DataFrame({
    'Métrica': ['R² Score', 'RMSE', 'MAE'],
    'Sin Record': [r2_sin_record, rmse_sin_record, mae_sin_record],
    'Con Record': [r2_con_record, rmse_con_record, mae_con_record]
})

comparacion['Diferencia'] = comparacion['Con Record'] - comparacion['Sin Record']
comparacion['Mejora (%)'] = (comparacion['Diferencia'] / comparacion['Sin Record'].abs()) * 100

print("="*80)
print("COMPARACIÓN DE MODELOS")
print("="*80)
print(comparacion.to_string(index=False))

print("\n" + "="*80)
print("ANÁLISIS DE MEJORA")
print("="*80)
if r2_con_record > r2_sin_record:
    mejora_r2 = ((r2_con_record - r2_sin_record) / r2_sin_record) * 100
    print(f"✓ El R² mejoró en {mejora_r2:.2f}% al incluir la variable Record")
else:
    print("✗ El R² NO mejoró al incluir la variable Record")

if rmse_con_record < rmse_sin_record:
    mejora_rmse = ((rmse_sin_record - rmse_con_record) / rmse_sin_record) * 100
    print(f"✓ El RMSE disminuyó en {mejora_rmse:.2f}% al incluir la variable Record")
else:
    print("✗ El RMSE NO mejoró al incluir la variable Record")

In [ ]:
# Visualización de predicciones vs valores reales
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Modelo sin Record
axes[0].scatter(y_test, y_pred_test_sr, alpha=0.5, edgecolors='k', linewidth=0.5)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('Valores Reales')
axes[0].set_ylabel('Predicciones')
axes[0].set_title(f'Modelo SIN Record (R² = {r2_sin_record:.4f})', fontweight='bold')
axes[0].grid(alpha=0.3)

# Modelo con Record
axes[1].scatter(y_test, y_pred_test_cr, alpha=0.5, edgecolors='k', linewidth=0.5, color='orange')
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1].set_xlabel('Valores Reales')
axes[1].set_ylabel('Predicciones')
axes[1].set_title(f'Modelo CON Record (R² = {r2_con_record:.4f})', fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Análisis de Multicolinealidad (VIF)

In [ ]:
# Calcular VIF para el modelo CON Record
vif_data = pd.DataFrame()
vif_data['Feature'] = features_con_record
vif_data['VIF'] = [variance_inflation_factor(X_con_record.values, i) 
                   for i in range(len(features_con_record))]
vif_data = vif_data.sort_values('VIF', ascending=False)

print("="*60)
print("ANÁLISIS DE MULTICOLINEALIDAD (VIF)")
print("="*60)
print("\nValores VIF (Variance Inflation Factor):")
print(vif_data.to_string(index=False))
print("\nInterpretación:")
print("VIF < 5:  No hay multicolinealidad preocupante")
print("VIF 5-10: Multicolinealidad moderada")
print("VIF > 10: Multicolinealidad alta (problemática)")

# Visualización
plt.figure(figsize=(10, 6))
colors = ['red' if x > 10 else 'orange' if x > 5 else 'green' for x in vif_data['VIF']]
plt.barh(vif_data['Feature'], vif_data['VIF'], color=colors, edgecolor='black')
plt.xlabel('VIF')
plt.title('Factor de Inflación de Varianza (VIF) por Variable', fontweight='bold')
plt.axvline(x=5, color='orange', linestyle='--', linewidth=1, label='VIF = 5')
plt.axvline(x=10, color='red', linestyle='--', linewidth=1, label='VIF = 10')
plt.legend()
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

## 8. Análisis de Residuos

In [ ]:
# Calcular residuos para ambos modelos
residuos_sr = y_test - y_pred_test_sr
residuos_cr = y_test - y_pred_test_cr

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Residuos vs Predicciones - Sin Record
axes[0, 0].scatter(y_pred_test_sr, residuos_sr, alpha=0.5, edgecolors='k', linewidth=0.5)
axes[0, 0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Predicciones')
axes[0, 0].set_ylabel('Residuos')
axes[0, 0].set_title('Residuos vs Predicciones (SIN Record)', fontweight='bold')
axes[0, 0].grid(alpha=0.3)

# Distribución de residuos - Sin Record
axes[0, 1].hist(residuos_sr, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Residuos')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_title('Distribución de Residuos (SIN Record)', fontweight='bold')
axes[0, 1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[0, 1].grid(alpha=0.3)

# Residuos vs Predicciones - Con Record
axes[1, 0].scatter(y_pred_test_cr, residuos_cr, alpha=0.5, edgecolors='k', 
                   linewidth=0.5, color='orange')
axes[1, 0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Predicciones')
axes[1, 0].set_ylabel('Residuos')
axes[1, 0].set_title('Residuos vs Predicciones (CON Record)', fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# Distribución de residuos - Con Record
axes[1, 1].hist(residuos_cr, bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1, 1].set_xlabel('Residuos')
axes[1, 1].set_ylabel('Frecuencia')
axes[1, 1].set_title('Distribución de Residuos (CON Record)', fontweight='bold')
axes[1, 1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Estadísticas de residuos
print("Estadísticas de Residuos:")
print("\nModelo SIN Record:")
print(f"Media: {residuos_sr.mean():.4f}")
print(f"Desviación estándar: {residuos_sr.std():.4f}")
print(f"Min: {residuos_sr.min():.4f}")
print(f"Max: {residuos_sr.max():.4f}")

print("\nModelo CON Record:")
print(f"Media: {residuos_cr.mean():.4f}")
print(f"Desviación estándar: {residuos_cr.std():.4f}")
print(f"Min: {residuos_cr.min():.4f}")
print(f"Max: {residuos_cr.max():.4f}")

## 9. Identificación de Variables con Impacto Negativo

In [ ]:
# Identificar variables con coeficientes negativos
print("="*80)
print("ANÁLISIS DE VARIABLES CON IMPACTO NEGATIVO")
print("="*80)

variables_negativas = coef_df_cr[coef_df_cr['Coefficient'] < 0]

if len(variables_negativas) > 0:
    print("\nVariables con coeficientes NEGATIVOS:")
    print(variables_negativas.to_string(index=False))
    
    print("\n" + "-"*80)
    print("INTERPRETACIÓN:")
    print("-"*80)
    for idx, row in variables_negativas.iterrows():
        print(f"\n• {row['Feature']}:")
        print(f"  Coeficiente: {row['Coefficient']:.4f}")
        print(f"  Por cada unidad que aumenta {row['Feature']}, ")
        print(f"  el Total Spend disminuye en ${abs(row['Coefficient']):.4f}")
else:
    print("\nNo se encontraron variables con coeficientes negativos.")

In [ ]:
# Análisis adicional: correlación de variables negativas con Total Spend
if len(variables_negativas) > 0:
    print("\nCorrelación directa de variables negativas con Total Spend:")
    for idx, row in variables_negativas.iterrows():
        feature = row['Feature']
        if feature in df.columns:
            corr = df[feature].corr(df['Total Spend'])
            print(f"{feature}: {corr:.4f}")

## 10. Modelo Optimizado (Eliminando variables problemáticas)

In [ ]:
# Crear modelo eliminando variables con coeficientes muy negativos o VIF alto
# Basado en el análisis anterior

# Identificar variables a eliminar (VIF > 10 o coeficiente muy negativo)
variables_problematicas = vif_data[vif_data['VIF'] > 10]['Feature'].tolist()
coef_muy_negativos = coef_df_cr[coef_df_cr['Coefficient'] < -100]['Feature'].tolist()

variables_a_eliminar = list(set(variables_problematicas + coef_muy_negativos))

print("Variables identificadas como problemáticas:")
print(variables_a_eliminar)

if len(variables_a_eliminar) > 0:
    features_optimizadas = [f for f in features_con_record if f not in variables_a_eliminar]
    
    print(f"\nFeatures para modelo optimizado: {features_optimizadas}")
    
    X_optimizado = df[features_optimizadas]
    X_train_opt, X_test_opt, y_train, y_test = train_test_split(
        X_optimizado, y, test_size=0.2, random_state=42
    )
    
    modelo_optimizado = LinearRegression()
    modelo_optimizado.fit(X_train_opt, y_train)
    
    y_pred_opt = modelo_optimizado.predict(X_test_opt)
    
    print("\n" + "="*60)
    print("MODELO OPTIMIZADO")
    print("="*60)
    print(f"R² Score: {r2_score(y_test, y_pred_opt):.4f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_opt)):.4f}")
    print(f"MAE: {mean_absolute_error(y_test, y_pred_opt):.4f}")
else:
    print("\nNo se identificaron variables altamente problemáticas para eliminar.")

## 11. Conclusiones y Respuestas

### Respuesta a Pregunta 1: ¿Qué pasaría si integramos la variable "Record" al modelo de predicción? ¿Mejoraría con esto la bondad de ajuste del modelo?

**ANÁLISIS:**

Basándonos en los resultados obtenidos:

1. **Comparación de R² (Coeficiente de Determinación):**
   - Modelo SIN Record: R² = [valor calculado]
   - Modelo CON Record: R² = [valor calculado]
   - Diferencia: [mejora o deterioro]

2. **Comparación de RMSE (Error Cuadrático Medio):**
   - Modelo SIN Record: RMSE = [valor]
   - Modelo CON Record: RMSE = [valor]
   - Un RMSE menor indica mejor precisión

3. **Coeficiente de la variable Record:**
   - El coeficiente indica el impacto de cada unidad de cambio en Record sobre Total Spend
   - Si es positivo: aumentar Record incrementa el gasto
   - Si es negativo: aumentar Record disminuye el gasto

**CONCLUSIÓN:**

[La conclusión se generará automáticamente basada en los resultados. Por ejemplo:]
- Si R² aumenta: "SÍ, la variable Record mejora la bondad de ajuste del modelo en un X%"
- Si R² disminuye o se mantiene igual: "NO, la variable Record no aporta mejora significativa al modelo"

**CONSIDERACIONES ADICIONALES:**
- Evaluar si el incremento en R² justifica la complejidad adicional del modelo
- Verificar si existe multicolinealidad con otras variables (VIF)
- Analizar si la variable Record tiene sentido teórico en el contexto del negocio

---

### Respuesta a Pregunta 2: ¿Existe alguna variable que pueda estar impactando en forma negativa las predicciones de nuestro modelo?

**ANÁLISIS DE VARIABLES CON IMPACTO NEGATIVO:**

1. **Variables con coeficientes negativos:**
   - [Lista de variables con coeficientes < 0]
   - Interpretación: estas variables tienen una relación inversa con Total Spend

2. **Análisis de multicolinealidad (VIF):**
   - Variables con VIF > 10 indican problemas severos de multicolinealidad
   - [Lista de variables problemáticas]

3. **Impacto en las predicciones:**
   - Coeficientes negativos no necesariamente son "malos" - pueden reflejar relaciones reales
   - El problema surge cuando:
     * El coeficiente negativo no tiene sentido en el contexto del negocio
     * Existe alta multicolinealidad
     * Los residuos muestran patrones sistemáticos

**VARIABLES IDENTIFICADAS COMO PROBLEMÁTICAS:**

[Basado en los resultados, se listarán las variables que:]:
1. Tienen VIF > 10 (alta multicolinealidad)
2. Tienen coeficientes negativos sin justificación teórica
3. Contribuyen a aumentar el error del modelo

**RECOMENDACIONES:**

1. **Transformación de variables:**
   - Considerar transformaciones logarítmicas o polinomiales
   - Normalización o estandarización de features

2. **Ingeniería de características:**
   - Crear interacciones entre variables relevantes
   - Eliminar o combinar variables con alta multicolinealidad

3. **Validación con expertos del negocio:**
   - Confirmar que las relaciones identificadas tienen sentido práctico
   - Considerar variables contextuales adicionales

4. **Técnicas alternativas:**
   - Regularización (Ridge, Lasso) para manejar multicolinealidad
   - Modelos no lineales si las relaciones no son lineales

---

**RESUMEN EJECUTIVO:**

Este análisis ha evaluado exhaustivamente el impacto de la variable Record en el modelo de regresión lineal y ha identificado potenciales variables problemáticas. Las métricas clave (R², RMSE, MAE) proporcionan evidencia cuantitativa para la toma de decisiones, mientras que el análisis de multicolinealidad (VIF) y los coeficientes del modelo ofrecen insights sobre la estructura y calidad del modelo predictivo.

---

**Autor:** Agustín Villarreal  
**Programa:** Maestría en Cómputo Aplicado  
**Email:** agustin.villarreal0743@alumnos.udg.mx  
**Fecha:** Octubre 2025